# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [2]:
view_line_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

In [3]:
lines[0]

'jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! you wanna go to sleep, you wanna get up, you wanna go out again tomorrow, right? where ever you are in life, its my feeling, youve gotta go. '

In [4]:
lines[1]

''

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [5]:
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    word_counts = Counter(text)
    sorted_vocabs = sorted(word_counts, key=word_counts.get, reverse=True)
    
    vocab_to_int = {word: i for i, word in enumerate(sorted_vocabs)}
    int_to_vocab = {i: word for i, word in enumerate(sorted_vocabs)}
    
    # return tuple
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [6]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    token_dict = {
        '.':  '||period||', 
        ',':  '||comma||', 
        '"':  '||quotation_mark||', 
        ';':  '||semicolon||', 
        '!':  '||exclamation_mark||', 
        '?':  '||question_mark||', 
        '(':  '||left_parentheses||', 
        ')':  '||right_parentheses||', 
        '-':  '||dash||', 
        '\n': '||return||'
    }
    
    return token_dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to lok at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [8]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

In [9]:
int_text[0:10]

[24, 22, 47, 1, 1, 1, 17, 47, 22, 82]

In [10]:
[int_to_vocab[word_int] for word_int in int_text[0:10]]

['this',
 'is',
 'out',
 '||period||',
 '||period||',
 '||period||',
 'and',
 'out',
 'is',
 'one']

In [11]:
[vocab_to_int[word] for word in 
    ['this',
     'is',
     'out',
     '||period||',
     '||period||',
     '||period||',
     'and',
     'out',
     'is',
     'one']]

[24, 22, 47, 1, 1, 1, 17, 47, 22, 82]

In [12]:
token_dict

{'.': '||period||',
 ',': '||comma||',
 '"': '||quotation_mark||',
 ';': '||semicolon||',
 '!': '||exclamation_mark||',
 '?': '||question_mark||',
 '(': '||left_parentheses||',
 ')': '||right_parentheses||',
 '-': '||dash||',
 '\n': '||return||'}

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [13]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

In [14]:
train_on_gpu

True

## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [15]:
from torch.utils.data import TensorDataset, DataLoader


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    # TODO: Implement function
    total_words = len(words)
    # trim total words to fit batch_size. plus 1 for the target_tensor
    total_words_fit =  total_words - ((total_words - 1) % sequence_length) + 1

    feature = []
    target = []
    for i in range(total_words_fit - (sequence_length + 1)):
        feature.append(words[i:i+sequence_length])
        target.append(words[i+sequence_length])
    
    feature_tensors = torch.IntTensor(feature)
    target_tensors = torch.IntTensor(target)
    data = TensorDataset(feature_tensors, target_tensors)
    data_loader = torch.utils.data.DataLoader(data, 
                                              batch_size=batch_size)
    
    # return a dataloader
    return data_loader

# there is no test for this function, but you are encouraged to create
# print statements and tests of your own


### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [16]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[ 0,  1,  2,  3,  4],
        [ 1,  2,  3,  4,  5],
        [ 2,  3,  4,  5,  6],
        [ 3,  4,  5,  6,  7],
        [ 4,  5,  6,  7,  8],
        [ 5,  6,  7,  8,  9],
        [ 6,  7,  8,  9, 10],
        [ 7,  8,  9, 10, 11],
        [ 8,  9, 10, 11, 12],
        [ 9, 10, 11, 12, 13]], dtype=torch.int32)

torch.Size([10])
tensor([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14], dtype=torch.int32)


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [17]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        # TODO: Implement function        
        # set class variables
        self.vocab_size = vocab_size
        self.output_size = output_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.dropout = dropout
        
        # define model layers
        self.embedding_layer = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.lstm_layer = nn.LSTM(self.embedding_dim, self.hidden_dim, self.n_layers, 
                                  dropout=self.dropout, batch_first=True)
        self.dropout_layer = nn.Dropout(self.dropout)
        self.fc_layer = nn.Linear(self.hidden_dim, self.output_size)
    
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        # TODO: Implement function
        batch_size = nn_input.size(0)
        
        nn_output = self.embedding_layer(nn_input.long())
        nn_output, hidden = self.lstm_layer(nn_output, hidden)
        nn_output = nn_output.contiguous().view(-1, self.hidden_dim)
        nn_output = self.dropout_layer(nn_output)
        nn_output = self.fc_layer(nn_output)
        nn_output = nn_output.view(batch_size, -1, self.output_size)

        # return one batch of output word scores and the hidden state
        return nn_output[:, -1], hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        
        # initialize hidden state with zero weights, and move to GPU if available
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [18]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param decoder: The PyTorch Module that holds the neural network
    :param decoder_optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # TODO: Implement Function
    
    # move data to GPU, if available
    if(train_on_gpu):
        inp, target = inp.cuda(), target.cuda()
    
    # perform backpropagation and optimization
    
    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    hidden = tuple([each_hidden.data for each_hidden in hidden])
    # zero accumulated gradients
    rnn.zero_grad()
    # get the output from the model
    output, hidden = rnn(inp, hidden)
    
    # calculate the loss and perform backprop
    loss = criterion(output, target.long())
    loss.backward()
    optimizer.step()

    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), hidden

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of uniqe tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [20]:
# Data params
# Sequence Length
sequence_length = 20  # of words in a sequence
# Batch Size
batch_size = 512

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [21]:
# Training parameters
# Number of Epochs
num_epochs = 60
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size (100K to 500K)
vocab_size = len(vocab_to_int)
print("vocab_size =", vocab_size)
# Output size (equal to input vocab size)
output_size = len(vocab_to_int)
print("output_size =", output_size)
# Embedding Dimension (100 to 1000. default 200)
embedding_dim = 200
# Hidden Dimension (200 to 1000. default 256)
hidden_dim = 256
# Number of RNN Layers (1 to 7. recommended 1 to 3. default 2 )
n_layers = 3

# Show stats for every n number of batches
show_every_n_batches = 500

vocab_size = 21388
output_size = 21388


### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [22]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 60 epoch(s)...
Epoch:    1/60    Loss: 5.917698300361633

Epoch:    1/60    Loss: 5.1994245262145995

Epoch:    1/60    Loss: 4.865314526557922

Epoch:    2/60    Loss: 4.650084992624679

Epoch:    2/60    Loss: 4.514176425457001

Epoch:    2/60    Loss: 4.437538671016693

Epoch:    3/60    Loss: 4.3496432805639715

Epoch:    3/60    Loss: 4.2956759042739865

Epoch:    3/60    Loss: 4.257341641902924

Epoch:    4/60    Loss: 4.195080964391765

Epoch:    4/60    Loss: 4.169544246673584

Epoch:    4/60    Loss: 4.145003530502319

Epoch:    5/60    Loss: 4.088757630628396

Epoch:    5/60    Loss: 4.078110851764679

Epoch:    5/60    Loss: 4.0615046906471255

Epoch:    6/60    Loss: 4.009921523438631

Epoch:    6/60    Loss: 4.004937076568604

Epoch:    6/60    Loss: 3.9915457625389097

Epoch:    7/60    Loss: 3.9414669345973956

Epoch:    7/60    Loss: 3.946706841945648

Epoch:    7/60    Loss: 3.9344761533737183

Epoch:    8/60    Loss: 3.8886626923501972

Epoch:    8/60    

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/lib/python3.6/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Dropout. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/lib/python3.6/site-pac

## Run 1

__Hyperparameters__
```
# Data params
# Sequence Length
sequence_length = 20  # of words in a sequence
# Batch Size
batch_size = 512

# Training parameters
# Number of Epochs
num_epochs = 10
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size (100K to 500K)
vocab_size = len(vocab_to_int)
print("vocab_size =", vocab_size)
# Output size (equal to input vocab size)
output_size = len(vocab_to_int)
print("output_size =", output_size)
# Embedding Dimension (100 to 1000. default 200)
embedding_dim = 200
# Hidden Dimension (200 to 1000. default 256)
hidden_dim = 256
# Number of RNN Layers (1 to 7. recommended 1 to 3. default 2 )
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 500
```

__GPU Utilization__
```
$  while sleep 0.2; do nvidia-smi --query-gpu=name,index,temperature.gpu,utilization.gpu,utilization.memory,memory.total,memory.free,memory.used --format=csv,noheader; done

GeForce GTX 1060, 0, 78, 98 %, 59 %, 6078 MiB, 2196 MiB, 3882 MiB
GeForce GTX 1060, 0, 78, 98 %, 58 %, 6078 MiB, 2196 MiB, 3882 MiB
GeForce GTX 1060, 0, 78, 98 %, 60 %, 6078 MiB, 2196 MiB, 3882 MiB
GeForce GTX 1060, 0, 78, 98 %, 56 %, 6078 MiB, 2196 MiB, 3882 MiB
GeForce GTX 1060, 0, 78, 99 %, 60 %, 6078 MiB, 2196 MiB, 3882 MiB
GeForce GTX 1060, 0, 78, 98 %, 55 %, 6078 MiB, 2196 MiB, 3882 MiB
GeForce GTX 1060, 0, 78, 98 %, 60 %, 6078 MiB, 2196 MiB, 3882 MiB
GeForce GTX 1060, 0, 78, 99 %, 61 %, 6078 MiB, 2196 MiB, 3882 MiB
GeForce GTX 1060, 0, 78, 99 %, 60 %, 6078 MiB, 2196 MiB, 3882 MiB
GeForce GTX 1060, 0, 78, 98 %, 57 %, 6078 MiB, 2196 MiB, 3882 MiB
GeForce GTX 1060, 0, 78, 98 %, 65 %, 6078 MiB, 2196 MiB, 3882 MiB
GeForce GTX 1060, 0, 78, 98 %, 57 %, 6078 MiB, 2196 MiB, 3882 MiB
GeForce GTX 1060, 0, 78, 98 %, 56 %, 6078 MiB, 2196 MiB, 3882 MiB
GeForce GTX 1060, 0, 78, 98 %, 59 %, 6078 MiB, 2196 MiB, 3882 MiB
GeForce GTX 1060, 0, 78, 98 %, 57 %, 6078 MiB, 2196 MiB, 3882 MiB
GeForce GTX 1060, 0, 78, 98 %, 58 %, 6078 MiB, 2196 MiB, 3882 MiB
GeForce GTX 1060, 0, 78, 98 %, 59 %, 6078 MiB, 2196 MiB, 3882 MiB
GeForce GTX 1060, 0, 78, 98 %, 58 %, 6078 MiB, 2196 MiB, 3882 MiB
```

__Loss History__
```
Training for 10 epoch(s)...
Epoch:    1/10    Loss: 5.559455227851868
Epoch:    1/10    Loss: 4.923607873916626
Epoch:    1/10    Loss: 4.7093134455680845
Epoch:    2/10    Loss: 4.510742139623493
Epoch:    2/10    Loss: 4.392335484981537
Epoch:    2/10    Loss: 4.336975105762482
Epoch:    3/10    Loss: 4.257968737430007
Epoch:    3/10    Loss: 4.211966034412384
Epoch:    3/10    Loss: 4.176790347576142
Epoch:    4/10    Loss: 4.120736593827404
Epoch:    4/10    Loss: 4.091817737579346
Epoch:    4/10    Loss: 4.0688235754966735
Epoch:    5/10    Loss: 4.019650522910681
Epoch:    5/10    Loss: 4.005084188938141
Epoch:    5/10    Loss: 3.988116059780121
Epoch:    6/10    Loss: 3.9387184333287157
Epoch:    6/10    Loss: 3.929239408493042
Epoch:    6/10    Loss: 3.916941858768463
Epoch:    7/10    Loss: 3.878308108553411
Epoch:    7/10    Loss: 3.8715184874534607
Epoch:    7/10    Loss: 3.8576121520996094
Epoch:    8/10    Loss: 3.8246136768808903
Epoch:    8/10    Loss: 3.819418927192688
Epoch:    8/10    Loss: 3.8101781706809996
Epoch:    9/10    Loss: 3.777212846311276
Epoch:    9/10    Loss: 3.7759403738975523
Epoch:    9/10    Loss: 3.7666970891952514
Epoch:   10/10    Loss: 3.737422841257162
Epoch:   10/10    Loss: 3.7391424989700317
Epoch:   10/10    Loss: 3.729654058456421
Model Trained and Saved
```

## Run 2

__Hyperparameters__
```
# Data params
# Sequence Length
sequence_length = 20  # of words in a sequence
# Batch Size
batch_size = 512

# Training parameters
# Number of Epochs
num_epochs = 60
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size (100K to 500K)
vocab_size = len(vocab_to_int)
print("vocab_size =", vocab_size)
# Output size (equal to input vocab size)
output_size = len(vocab_to_int)
print("output_size =", output_size)
# Embedding Dimension (100 to 1000. default 200)
embedding_dim = 200
# Hidden Dimension (200 to 1000. default 256)
hidden_dim = 256
# Number of RNN Layers (1 to 7. recommended 1 to 3. default 2 )
n_layers = 3

# Show stats for every n number of batches
show_every_n_batches = 500
```

__GPU Utilization__
```
GeForce GTX 1060, 0, 74, 98 %, 56 %, 6078 MiB, 2128 MiB, 3950 MiB
GeForce GTX 1060, 0, 74, 98 %, 59 %, 6078 MiB, 2128 MiB, 3950 MiB
GeForce GTX 1060, 0, 74, 98 %, 61 %, 6078 MiB, 2128 MiB, 3950 MiB
GeForce GTX 1060, 0, 74, 98 %, 61 %, 6078 MiB, 2128 MiB, 3950 MiB
GeForce GTX 1060, 0, 74, 98 %, 55 %, 6078 MiB, 2128 MiB, 3950 MiB
GeForce GTX 1060, 0, 74, 98 %, 60 %, 6078 MiB, 2128 MiB, 3950 MiB
GeForce GTX 1060, 0, 74, 98 %, 59 %, 6078 MiB, 2128 MiB, 3950 MiB
GeForce GTX 1060, 0, 74, 98 %, 57 %, 6078 MiB, 2128 MiB, 3950 MiB
GeForce GTX 1060, 0, 74, 98 %, 60 %, 6078 MiB, 2128 MiB, 3950 MiB
GeForce GTX 1060, 0, 74, 98 %, 56 %, 6078 MiB, 2128 MiB, 3950 MiB
GeForce GTX 1060, 0, 74, 98 %, 58 %, 6078 MiB, 2128 MiB, 3950 MiB
GeForce GTX 1060, 0, 74, 98 %, 56 %, 6078 MiB, 2128 MiB, 3950 MiB
GeForce GTX 1060, 0, 74, 98 %, 56 %, 6078 MiB, 2129 MiB, 3949 MiB
GeForce GTX 1060, 0, 74, 98 %, 60 %, 6078 MiB, 2129 MiB, 3949 MiB
GeForce GTX 1060, 0, 74, 98 %, 56 %, 6078 MiB, 2129 MiB, 3949 MiB
GeForce GTX 1060, 0, 74, 98 %, 56 %, 6078 MiB, 2129 MiB, 3949 MiB
GeForce GTX 1060, 0, 74, 98 %, 56 %, 6078 MiB, 2129 MiB, 3949 MiB
GeForce GTX 1060, 0, 74, 98 %, 62 %, 6078 MiB, 2129 MiB, 3949 MiB
```
__Loss History__
```
Training for 60 epoch(s)...
Epoch:    1/60    Loss: 5.925266566276551
Epoch:    1/60    Loss: 5.193898889541626
Epoch:    1/60    Loss: 4.837475499153137
Epoch:    2/60    Loss: 4.621767043424745
Epoch:    2/60    Loss: 4.494452564239502
Epoch:    2/60    Loss: 4.425045051574707
Epoch:    3/60    Loss: 4.34227283238722
Epoch:    3/60    Loss: 4.289236557006836
Epoch:    3/60    Loss: 4.245486149787903
Epoch:    4/60    Loss: 4.191003017991059
Epoch:    4/60    Loss: 4.161075520992279
Epoch:    4/60    Loss: 4.132576570034027
Epoch:    5/60    Loss: 4.086390723757988
Epoch:    5/60    Loss: 4.070616644859314
Epoch:    5/60    Loss: 4.048916274547577
Epoch:    6/60    Loss: 4.004725973239806
Epoch:    6/60    Loss: 3.990745204925537
Epoch:    6/60    Loss: 3.9783622155189513
Epoch:    7/60    Loss: 3.9429361479622975
Epoch:    7/60    Loss: 3.930980802536011
Epoch:    7/60    Loss: 3.922416799068451
Epoch:    8/60    Loss: 3.8897702777482106
Epoch:    8/60    Loss: 3.8828969068527224
Epoch:    8/60    Loss: 3.8732625703811645
Epoch:    9/60    Loss: 3.8467253552614196
Epoch:    9/60    Loss: 3.8388537564277647
Epoch:    9/60    Loss: 3.833730870246887
Epoch:   10/60    Loss: 3.8091056285842733
Epoch:   10/60    Loss: 3.805123044490814
Epoch:   10/60    Loss: 3.800647054672241
Epoch:   11/60    Loss: 3.7749008544371776
Epoch:   11/60    Loss: 3.7733011384010315
Epoch:   11/60    Loss: 3.7661554851531984
Epoch:   12/60    Loss: 3.7473365434739145
Epoch:   12/60    Loss: 3.7423632164001464
Epoch:   12/60    Loss: 3.7400807375907896
Epoch:   13/60    Loss: 3.7179725729230277
Epoch:   13/60    Loss: 3.719324627876282
Epoch:   13/60    Loss: 3.7142025299072268
Epoch:   14/60    Loss: 3.6944974921141673
Epoch:   14/60    Loss: 3.696405525684357
Epoch:   14/60    Loss: 3.692813035964966
Epoch:   15/60    Loss: 3.671480292901196
Epoch:   15/60    Loss: 3.6736877036094664
Epoch:   15/60    Loss: 3.673105314254761
Epoch:   16/60    Loss: 3.6521021294786604
Epoch:   16/60    Loss: 3.652766338825226
Epoch:   16/60    Loss: 3.652897190093994
Epoch:   17/60    Loss: 3.6330612923578434
Epoch:   17/60    Loss: 3.634097620010376
Epoch:   17/60    Loss: 3.634902262210846
Epoch:   18/60    Loss: 3.6190319125543065
Epoch:   18/60    Loss: 3.6192986993789673
Epoch:   18/60    Loss: 3.6152473239898684
Epoch:   19/60    Loss: 3.5993573504959477
Epoch:   19/60    Loss: 3.6000866904258726
Epoch:   19/60    Loss: 3.5983353333473205
Epoch:   20/60    Loss: 3.5879160442120948
Epoch:   20/60    Loss: 3.5868926644325256
Epoch:   20/60    Loss: 3.5863179812431336
Epoch:   21/60    Loss: 3.5736215275252925
Epoch:   21/60    Loss: 3.5662837009429933
Epoch:   21/60    Loss: 3.5678228178024294
Epoch:   22/60    Loss: 3.5603189243460602
Epoch:   22/60    Loss: 3.553656003475189
Epoch:   22/60    Loss: 3.5549378538131715
Epoch:   23/60    Loss: 3.547778616696998
Epoch:   23/60    Loss: 3.5403094606399534
Epoch:   23/60    Loss: 3.538934359550476
Epoch:   24/60    Loss: 3.5330296333909357
Epoch:   24/60    Loss: 3.5263347392082216
Epoch:   24/60    Loss: 3.526570559978485
Epoch:   25/60    Loss: 3.5238775609317172
Epoch:   25/60    Loss: 3.5191599864959717
Epoch:   25/60    Loss: 3.517512766838074
Epoch:   26/60    Loss: 3.511567704439806
Epoch:   26/60    Loss: 3.5048704719543458
Epoch:   26/60    Loss: 3.5050519828796385
Epoch:   27/60    Loss: 3.50368115458527
Epoch:   27/60    Loss: 3.494914342403412
Epoch:   27/60    Loss: 3.4931777229309082
Epoch:   28/60    Loss: 3.492735995757933
Epoch:   28/60    Loss: 3.482516394138336
Epoch:   28/60    Loss: 3.4864102306365967
Epoch:   29/60    Loss: 3.4781503940849614
Epoch:   29/60    Loss: 3.472372728347778
Epoch:   29/60    Loss: 3.472203057765961
Epoch:   30/60    Loss: 3.473675138866805
Epoch:   30/60    Loss: 3.4644469509124756
Epoch:   30/60    Loss: 3.46553794670105
Epoch:   31/60    Loss: 3.4613179084104346
Epoch:   31/60    Loss: 3.4538368945121767
Epoch:   31/60    Loss: 3.457744989871979
Epoch:   32/60    Loss: 3.452024652630171
Epoch:   32/60    Loss: 3.4458408517837524
Epoch:   32/60    Loss: 3.446110300540924
Epoch:   33/60    Loss: 3.445629841555161
Epoch:   33/60    Loss: 3.435339813709259
Epoch:   33/60    Loss: 3.437936611175537
Epoch:   34/60    Loss: 3.4338118027484
Epoch:   34/60    Loss: 3.4283084921836853
Epoch:   34/60    Loss: 3.4264192299842833
Epoch:   35/60    Loss: 3.430187555336245
Epoch:   35/60    Loss: 3.4169768447875977
Epoch:   35/60    Loss: 3.424375892162323
Epoch:   36/60    Loss: 3.4171530281437055
Epoch:   36/60    Loss: 3.4077566623687745
Epoch:   36/60    Loss: 3.411784192562103
Epoch:   37/60    Loss: 3.4150305247371087
Epoch:   37/60    Loss: 3.399477096557617
Epoch:   37/60    Loss: 3.403841211795807
Epoch:   38/60    Loss: 3.4070196585513832
Epoch:   38/60    Loss: 3.3984809579849244
Epoch:   38/60    Loss: 3.396602550983429
Epoch:   39/60    Loss: 3.400738736368575
Epoch:   39/60    Loss: 3.387154908657074
Epoch:   39/60    Loss: 3.3851445875167845
Epoch:   40/60    Loss: 3.394760429377183
Epoch:   40/60    Loss: 3.3812872996330263
Epoch:   40/60    Loss: 3.3834554591178896
Epoch:   41/60    Loss: 3.3909063107883832
Epoch:   41/60    Loss: 3.3722584381103515
Epoch:   41/60    Loss: 3.3746827692985533
Epoch:   42/60    Loss: 3.3826734849384854
Epoch:   42/60    Loss: 3.3687995090484617
Epoch:   42/60    Loss: 3.371664252758026
Epoch:   43/60    Loss: 3.375947047757974
Epoch:   43/60    Loss: 3.364464599609375
Epoch:   43/60    Loss: 3.362281635761261
Epoch:   44/60    Loss: 3.3680647544141085
Epoch:   44/60    Loss: 3.354453498363495
Epoch:   44/60    Loss: 3.3515125513076782
Epoch:   45/60    Loss: 3.3649422434783687
Epoch:   45/60    Loss: 3.349566061973572
Epoch:   45/60    Loss: 3.3478255276679993
Epoch:   46/60    Loss: 3.35528622611835
Epoch:   46/60    Loss: 3.340983386993408
Epoch:   46/60    Loss: 3.342066213607788
Epoch:   47/60    Loss: 3.3567252422600102
Epoch:   47/60    Loss: 3.33699560880661
Epoch:   47/60    Loss: 3.333661367416382
Epoch:   48/60    Loss: 3.3491333048298673
Epoch:   48/60    Loss: 3.3293499760627747
Epoch:   48/60    Loss: 3.3274163784980773
Epoch:   49/60    Loss: 3.3402978194691744
Epoch:   49/60    Loss: 3.32772630405426
Epoch:   49/60    Loss: 3.324006186962128
Epoch:   50/60    Loss: 3.3386118948620283
Epoch:   50/60    Loss: 3.3167394666671752
Epoch:   50/60    Loss: 3.322242060661316
Epoch:   51/60    Loss: 3.333722508178567
Epoch:   51/60    Loss: 3.3150722661018373
Epoch:   51/60    Loss: 3.3165596685409544
Epoch:   52/60    Loss: 3.3294389495309793
Epoch:   52/60    Loss: 3.3062019066810606
Epoch:   52/60    Loss: 3.3064555697441103
Epoch:   53/60    Loss: 3.3230556741236357
Epoch:   53/60    Loss: 3.3079326672554017
Epoch:   53/60    Loss: 3.3030969853401184
Epoch:   54/60    Loss: 3.319050393335903
Epoch:   54/60    Loss: 3.3020467548370362
Epoch:   54/60    Loss: 3.2973513293266294
Epoch:   55/60    Loss: 3.321463287358657
Epoch:   55/60    Loss: 3.3006456480026247
Epoch:   55/60    Loss: 3.29261026763916
Epoch:   56/60    Loss: 3.311101557430874
Epoch:   56/60    Loss: 3.295542236804962
Epoch:   56/60    Loss: 3.290331736564636
Epoch:   57/60    Loss: 3.3132928879755847
Epoch:   57/60    Loss: 3.284811435699463
Epoch:   57/60    Loss: 3.2889635529518126
Epoch:   58/60    Loss: 3.3024958542415073
Epoch:   58/60    Loss: 3.2929654107093813
Epoch:   58/60    Loss: 3.2844894523620605
Epoch:   59/60    Loss: 3.2994948953309793
Epoch:   59/60    Loss: 3.2795934019088744
Epoch:   59/60    Loss: 3.2754066100120545
Epoch:   60/60    Loss: 3.2958736377906286
Epoch:   60/60    Loss: 3.2772466015815733
Epoch:   60/60    Loss: 3.2766857051849367
Model Trained and Saved
```

### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

**Answer:** (Write answer, here)

Many of hyperparameters I use are based on papers summary taken from Part 4, Lesson 4, Section 8 about RNN Hyperparameters. This includes LSTM model to use, number of RNN layers from 1 to 7 or 1 to 3 as recommended in the project rubric, RNN hidden dimension from 200 to 1000, embedding size/dimension of around 100 to 1000 and learning rate of 0.001.

Base consideration of above hyperparameters is making a bigger model (better learning capability) that still fit in my GPU (GTX 1060 with 6 GiB memory). I tried both 2 and 3 RNN layers. 256 RNN hidden dimension that is commonly used in the papers summary suitable for simple tasks. 200 for embedding dimension among 50, 128 and 250 commonly used in TensorFlow Hub. Since the entire words in the vocabulary is just slightly more than 21 thousand, 200 dimensions for the embedding should be sufficient. 250 embedding dimensions in TensorFlow Hub model collection is for more than 100 thousand words.

With common batch size of 512 that is large enough, the model takes about 4 GiB. Changing from 2 to 3 RNN layers only utilize extra 67 MiB of GPU memory from 3882 to 3949 MiB. In 10 epochs, 3 layers starts with higher loss but ends with comparable loss 3.8 and 3.7 for 3 and 2 layers respectively.

I use the entire 21,388 words from the vocabulary that is still quite small compared to 100 to 500 thousand words commonly used. This is applicable for both the input and the output of the model. I also use sequence length of 20 that is higher than most conversations so the model can learn from a context that flows to the next conversation.

I also tried both 10 and 60 epochs. The model can achieve lower than 3.5 of loss after 27 epochs.

---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [23]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [26]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq_device = torch.LongTensor(current_seq).cuda()
        else:
            current_seq_device = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq_device.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq_device, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [29]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

jerry:

jerry:(looking at his face) oh, i can't believe i could. i can't believe that you are a good man, i don't know, but i think it's all right.

george:(sarcastic) well, i can't believe i was just going to get a little problem, and i have to get a little nervous.

jerry:(to jerry) hey, you know what you are?

kramer: oh, i'm going to be a little bit.

elaine:(to kramer) i can't believe that was a good day.

elaine: oh yeah. i got a little trouble.

george: oh yeah, yeah, i'm a little tired-- i don't know what you do.

kramer:(confused) i thought you can have the hat.

kramer: oh.

jerry:(sarcastic) oh, that's right, i'm gonna have a new place to see.

elaine:(getting up) oh, no, no, no, no, no, no, no no! you can't.

kramer:(to jerry) i have a little good idea with that. you know, i don't have to be in a little bit of a little problem for the rest of the world.

kramer: hey, hey, jerry.

elaine:(to george) what is this?

kramer: yeah, it's a good day.

elaine: well, i'm sure i'm no

#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [30]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.